In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name = "KoalaAI/OffensiveSpeechDetector"

# Load model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Save to local directory
save_path = "./offensive_speech_model"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs).item()

    # Map class index to label
    labels = ["Not Offensive", "Offensive"]
    return labels[predicted_class], probs.squeeze().tolist()